In [1]:
db_link = 'sqlite:////home/ostapkharysh/Documents/bt_data/DB/news_info.db'

In [2]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
 
Base = declarative_base()
 
class Agency(Base):
    __tablename__ = 'agency'
    # define columns for the table agency
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
 
class News(Base):
    __tablename__ = 'news'
    # define columns for the table news.
    id = Column(Integer, primary_key=True)
    date_time = Column(String(250))
    title = Column(String(250))
    article_text = Column(String(1000))
    link = Column(String(250), unique=True)  #nullable=False
    agency_id = Column(Integer, ForeignKey('agency.id'))
    agency = relationship(Agency)
  
 
# Create an engine that stores data in the local directory's
engine = create_engine(db_link)
 
# Create all tables in the engine.
Base.metadata.create_all(engine)

In [11]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
#from 'filename' import Base, Agency, News


def add_agency(agency_name):
    
    engine = create_engine(db_link)
    # Bind the engine to the metadata of the Base class so that the
    # declaratives can be accessed through a DBSession instance
    Base.metadata.bind = engine
     
    DBSession = sessionmaker(bind=engine)
    
    # A DBSession() instance establishes all conversations with the database
    # and represents a "staging zone" for all the objects loaded into the
    # database session object. Any change made against the objects in the
    # session won't be persisted into the database until you call
    # session.commit(). If you're not happy about the changes, you can
    # revert all of them back to the last commit by calling
    # session.rollback()
    
    session = DBSession()
    
    data = session.query(Agency).all()
    if agency_name in [el.name for el in data]:
        return "There is already a Table with such name: {}".format(agency_name)

 
    # Insert a Agency in the agency table
    new_agency = Agency(name=agency_name)
    session.add(new_agency)
    session.commit()
    return "The new table {} is created.".format(agency_name)
    
def add_news(d_t, ttl, ar_txt, lnk, agency_name):
    
    # Insert an news in the address table
    engine = create_engine(db_link)
    Base.metadata.bind = engine
    DBSession = sessionmaker(bind=engine)
    session = DBSession()
    
    try:
        
        cur_agency = session.query(Agency).filter_by(name=agency_name).first()
        new_news = News(date_time=d_t, title= ttl, article_text=ar_txt, link=lnk, agency=cur_agency)
        session.add(new_news)
        session.commit()
        
    except IntegrityError:
        session.rollback()
        return 'The link provided seems to exist in DB: {}'.format(lnk)
    
    except InvalidRequestError:
        session.rollback()
        return 'You are requesting access to the non-existing source'
    
    
        
    return "The news has been successfully added"

In [6]:
print(add_agency('Reuters'))
print(add_agency('Guardian'))

There is already a Table with such name: Reuters
There is already a Table with such name: Guardian


In [13]:
add_news('2018.01.01', "A Manhattan bitch stole the deer from the client's flat",'There was a time when bitches were rulling the world, but this time has passed' 
                ,'https/ivent-zi-vsratou-nasvou-na-iaku-vsi-pidpusuiutsia', 'Reuters') 

'The link provided seems to exist in DB: https/ivent-zi-vsratou-nasvou-na-iaku-vsi-pidpusuiutsia'

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [6]:
engine = create_engine(db_link)
Base.metadata.bind = engine

In [21]:
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()
# Make a query to find all Persons in the database
d = session.query(Agency).all()
n = [el.name for el in d]

In [28]:
session.query(Agency).filter_by(name='Reuters').first() 

In [11]:
# Return the first Person from all Persons in the database
agency = session.query(Agency).first()
agency.name

AttributeError: 'Query' object has no attribute 'name'

In [9]:
# Find all Address whose person field is pointing to the person object
session.query(News).filter(News.agency == agency).all()

In [10]:
news = session.query(News).filter(News.agency == agency).one()
news.date_time

'2018.01.01'